# Path of Exile game stats data eda

dates: 2022.04.25 ~ 

This file contains an analysis processing of the game 'Path of Exile' data.

###### Origins of datas from here: 
poe_stats : https://www.kaggle.com/datasets/gagazet/path-of-exile-league-statistic

Data contains stats of 59000 players, from 4th August of 2017 and before now.

---

This file made for EDA after pre-processing the original data, arbitrarily setted by me.

본격적인 EDA는 이 파일에서 시작하려고 합니다. 첫 EDA 주제는 랭크에 따라 분석을 하는 것이며, 

15,000 등 안에 드는 이 랭커들의 직업이나 경험치, 직업과 경험치 등을 살펴볼 예정입니다.

그 다음 전체적인 분석과 각 게임 모드 별로, 그리고 한 사람이 여러 계정을 쓰는 경우 등등을 앞으로 볼 것입니다.

# Dataset and libraries 

## data and libraries import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from scipy.stats import trim_mean
from statsmodels import robust
import wquantiles

df = pd.read_csv('poe_stats_1preprocessing.csv')

In [2]:
df

,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
0,1,False,False,Tzn_NecroIsFineNow,100,Necromancer,3dcddd59f5088893f734f39686350990dae168cc4f4b00...,4250334444,TheTzn,40,thetzn,Harbinger,1,0,1,0,1
1,1,False,False,RaizNeverFirstQT,100,Necromancer,8f3216db5ac9106c287a834731aafc83c387138f28fc45...,4250334444,RaizQT,22,raizqt,SSF Harbinger HC,1,0,1,0,1
2,1,False,False,GucciStreamerAdvantage,100,Necromancer,c6ec2dae3855c551e0597c06ef2da06fbb5512487ded7f...,4250334444,GucciPradas,35,guccipradas,Hardcore Harbinger,1,0,1,0,1
3,1,False,False,ChiroxPrime,100,Slayer,c861372da792be0b22c45bf437ccd58437c52e9455e0bf...,4250334444,Chiroxun,26,NaN,SSF Harbinger,1,1,0,1,2
4,2,False,False,Cool_NecroIsFineNow,100,Deadeye,24ae924ceed7989ef3d3d6772612832bb467a6094352c7...,4250334444,cooltail,22,NaN,Harbinger,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59771,14999,True,False,ПроклятьеРекласта,89,Necromancer,d33b4f6e08c10e365765f9a36a8f36d561fd1d86f10ee1...,1786695653,Romanchik002,18,romanchik002,Hardcore Harbinger,1,0,1,0,1
59772,15000,False,False,IshibashiSummoner,94,Necromancer,5764cfa387e0a87a4bebc1a3c5017e92de8bbb06445c47...,2720770853,Ishibashisan,33,NaN,Harbinger,1,0,1,0,1
59773,15000,False,False,BLively,73,Slayer,9ac75ab75a47cee8a9dfb0a31912df8909720a8b20f9f1...,495034029,ruby01,19,NaN,SSF Harbinger,1,1,0,1,2
59774,15000,False,False,vawddvaw,89,Gladiator,cf02dfc0c90b2df9c7ac76bbedd91e93c2a8a2ca629c15...,1786638972,Lecarn,24,lecarn,Hardcore Harbinger,1,1,0,1,2


In [3]:
df.shape

(59776, 17)

In [4]:
df.sample(10)

,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
46607,11693,True,False,Brazilian,90,Juggernaut,5e74d293893d28e88fc09896615e6b60561197ad58f4ae...,1934009687,MatheusGirardelli,25,NaN,Hardcore Harbinger,1,1,0,0,1
32445,8134,False,False,NecroNugg,82,Necromancer,a60e5ceeaa46460713fde89708735b17d979b43d0ea86a...,1050972369,Jackaryus,17,NaN,SSF Harbinger,1,0,1,0,1
39965,10022,False,False,TumeniPlectraThrows,95,Pathfinder,5edfc32c39eccf2f0366a8b5d2488017345788ecbbe261...,2880421893,scitscatwah3,36,NaN,Harbinger,1,0,0,1,1
57594,14453,False,False,BDSM,74,Necromancer,2eb7fd69b4b6a9dd947114a86c85746a58a48890b595f2...,520980620,Lahmatyj,12,NaN,SSF Harbinger,1,0,1,0,1
26947,6754,False,False,gooberboobbutt,95,Guardian,3266be5bfc12751b82c69de0f52b67331dabdfb9f47b1a...,2983991187,ivanius,29,NaN,Harbinger,1,1,1,0,2
308,78,False,False,Vteccer,96,Gladiator,38e16066e3d9e3da167a992ecc9ac7d50221f63e43f5fe...,3121027276,Masterofpupetz,22,NaN,SSF Harbinger,1,1,0,1,2
22845,5726,True,False,Robs_IceIceBaby,92,Raider,1f165f52ce79d99f1f23d53450e07bdc089046ed8de48a...,2268660435,ruustas,36,roztar,Hardcore Harbinger,1,0,0,1,1
36149,9063,True,False,Aimag,68,Ascendant,ef6002f87e0ae820619f7ab2de95e34e6fb2a894fb9037...,323639678,Fomalhaut,12,NaN,SSF Harbinger HC,1,1,1,1,3
45921,11520,False,False,Xani_Wanderella,95,Pathfinder,b44e66c3bec1894564c0d22ae06f76a049b4b502a1eed7...,2876768314,xanityz,40,xanity_poe,Harbinger,1,0,0,1,1
59288,14878,True,False,TheNchtA,53,Necromancer,4186630298fc4002bd1d5382b7b3a0b2e8a2285297852d...,79783860,Ne4tA,13,NaN,SSF Harbinger HC,1,0,1,0,1


## data describes

In [5]:
# columns name

df.columns

Index(['rank', 'dead', 'online', 'name', 'level', 'class', 'id', 'experience',
       'account', 'challenges', 'twitch', 'ladder', 'special_class',
       'stat_Str', 'stat_Int', 'stat_Dex', 'stats_num'],
      dtype='object')

In [6]:
df.describe()

,rank,level,experience,challenges,special_class,stat_Str,stat_Int,stat_Dex,stats_num
count,59776.000000,59776.000000,5.977600e+04,59776.000000,59776.000000,59776.000000,59776.000000,59776.000000,59776.000000
mean,7495.369379,85.422578,1.825786e+09,25.759569,0.991702,0.582207,0.418027,0.513450,1.513684
std,4330.179898,11.288017,1.094748e+09,8.440888,0.090713,0.493200,0.493239,0.499823,0.614134
min,1.000000,53.000000,7.790982e+07,4.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,3745.000000,78.000000,7.774697e+08,19.000000,1.000000,0.000000,0.000000,0.000000,1.000000
50%,7492.000000,90.000000,1.934010e+09,25.000000,1.000000,1.000000,0.000000,1.000000,1.000000
75%,11247.000000,95.000000,2.876117e+09,33.000000,1.000000,1.000000,1.000000,1.000000,2.000000
max,15000.000000,100.000000,4.250334e+09,40.000000,1.000000,1.000000,1.000000,1.000000,3.000000


In [7]:
df.describe(include='object')

,name,class,id,account,twitch,ladder
count,59776,59776,59776,59776,16300,59776
unique,58286,26,59776,37123,9613,4
top,_Flic_,Berserker,3dcddd59f5088893f734f39686350990dae168cc4f4b00...,chuz101,thespunkylobster,SSF Harbinger
freq,14,8711,1,30,18,14981


In [8]:
df.nunique()

rank             15000
dead                 2
online               2
name             58286
level               48
class               26
id               59776
experience       54124
account          37123
challenges          37
twitch            9613
ladder               4
special_class        2
stat_Str             2
stat_Int             2
stat_Dex             2
stats_num            3
dtype: int64

In [9]:
df.isnull().sum()

rank                 0
dead                 0
online               0
name                 0
level                0
class                0
id                   0
experience           0
account              0
challenges           0
twitch           43476
ladder               0
special_class        0
stat_Str             0
stat_Int             0
stat_Dex             0
stats_num            0
dtype: int64

### describe results

데이터 총 개수 59776 개

컬럼명: ['rank', 'dead', 'online', 'name', 'level', 'class', 'id', 'experience', 'account', 'challenges', 'twitch', 'ladder']

랭크는 총 1만 5천개, 죽음과 온라인 여부는 True, False로 이루어져있다.

id가 오직 unique 하며, 한 아이디당 여러 account를 가질 수 있는 것으로 보인다. 그에 따라 일부 name은 겹치는 것을 보인다.

level 과 class는 게임 내 레벨과 직업으로 이루어져 있다.

experience 는 게임 내 경험치를 의미하는 것일 것이며,

twitch는 트위치 계정이 적혀있는 컬럼으로 보인다. 일부 살펴본 결과 한 트위치 아이디로 여러 id를 운영하며 스트리밍 하는 것으로 보인다. 최대 18 아이디를 운영하는 twitch id도 있다.

challenges 의 경우 poe 게임이 던전형 rpg임을 감안하여 짐작해보건대 해당 랭크를 따기까지 해당 모드를 몇 번 플레이 했는지를 나타내는 것으로 예상된다.

ladder 의 경우 플레이모드로, SSF는 솔로 모드를 뜻하고 Hardcore = HC 로 추정된다.


### in addition

*분석의 목적을 정확히 정하고 싶지만 이 게임에서 랭킹에 드는 기준조차 제대로 모르기 때문에, 랭커들의 직업이나 특성들을 찾아보는 것을 주 목적으로 하되, 데이터로 게임(맥락)을 이해할 수 있을지 궁금한 부분도 있어서 최대한 많은 생각을 하면서 분석을 진행하려 한다.*

## EDA planning

이에 따라 현 데이터 셋은 크게 이렇게 나누어 볼 수 있을 것이다.

1. rank (rank bins) 
2. level or exp bins
3. class or stats
4. ladder

또한 위의 내용을 laddar 로 나누어 다시 나누어 볼 수 있을 것이다.

## But first, I think we should look by ladder (game mode)

1. 게임 모드별로 나눈다.

In [16]:
df['ladder'].describe()

count             59776
unique                4
top       SSF Harbinger
freq              14981
Name: ladder, dtype: object

In [21]:
df['rank'].value_counts()

1        4
9950     4
9938     4
9939     4
9940     4
        ..
13443    3
3314     3
1506     3
6477     3
2367     2
Name: rank, Length: 15000, dtype: int64

In [26]:
df['rank'].value_counts().max()

4

각 랭크는 최대 4명으로 되어있다. 이것만 보아도 게임 모드에 따라 랭크가 구분되어 있음을 미루어 짐작해 볼 수 있다. 확인해본다.

In [29]:
pd.pivot_table(data = df, index = ['ladder'], aggfunc = 'count')[['account', 'rank']]

,account,rank
ladder,,
Harbinger,14918,14918
Hardcore Harbinger,14905,14905
SSF Harbinger,14981,14981
SSF Harbinger HC,14972,14972


아이디의 개수와 랭크의 개수가 일치하는 것으로 나왔다. 게임 모드에 따라 랭크가 구분되어 있음이 맞다.

개인적인 생각으로, 이 데이터는 rank별로 나열될 것이 아니라, ladder 별로 rank가 나열되어야 하는 것이 맞다고 생각이 된다.



In [51]:
# 각 모드의 id, rank 고유 개수

modes = ['Harbinger', 'Hardcore Harbinger', 'SSF Harbinger', 'SSF Harbinger HC']

for i in modes:
    print("ladder: ", i)
    print("values of unique id", df[df['ladder'] == i]['id'].nunique())
    print("values of unique rank", df[df['ladder'] == i]['rank'].nunique())
    print('-------------------------------')

ladder:  Harbinger
values of unique id 14918
values of unique rank 14918
-------------------------------
ladder:  Hardcore Harbinger
values of unique id 14905
values of unique rank 14905
-------------------------------
ladder:  SSF Harbinger
values of unique id 14981
values of unique rank 14981
-------------------------------
ladder:  SSF Harbinger HC
values of unique id 14972
values of unique rank 14972
-------------------------------


In [30]:
df[df['ladder'] == 'Harbinger']['rank']

0            1
4            2
8            3
13           4
18           5
         ...  
59757    14996
59761    14997
59766    14998
59770    14999
59772    15000
Name: rank, Length: 14918, dtype: int64

분명 15000 등까지 있는 것으로 나타나지만 nunique를 넣어보면 팔십 여개가 사라져 있다.

수가 더 적기 때문에 동점가 있는 것 같지는 않고, 크롤링 상의 문제이거나, 내부 데이터 상의 어떤 그런 것에 의해서 일부 없는 것으로 생각된다. (게임모드에서는 결측치가 없었기 때문)

등수는 절대적인 수치가 아니고, 나열 되어서 순서가 있을 때 의미가 존재하기 때문에 없는 등수를 채워서 총 14918 등까지 나타나도록 수정할 것이다.

## df divising

In [40]:
df_Har = df[df['ladder'] == 'Harbinger']
df_HarHC = df[df['ladder'] == 'Hardcore Harbinger']
df_SHar = df[df['ladder'] == 'SSF Harbinger']
df_SHarHC = df[df['ladder'] == 'SSF Harbinger HC']

In [41]:
df_Har

,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
0,1,False,False,Tzn_NecroIsFineNow,100,Necromancer,3dcddd59f5088893f734f39686350990dae168cc4f4b00...,4250334444,TheTzn,40,thetzn,Harbinger,1,0,1,0,1
4,2,False,False,Cool_NecroIsFineNow,100,Deadeye,24ae924ceed7989ef3d3d6772612832bb467a6094352c7...,4250334444,cooltail,22,NaN,Harbinger,1,0,0,1,1
8,3,False,False,Rami_The_Negromaniac,100,Necromancer,00c0030fc548cc4c0d804564f8fb356610044b7e6d484f...,4250334444,rami1337,40,rami1337,Harbinger,1,0,1,0,1
13,4,False,False,Guraj_NecroIsFineNow,100,Raider,34bedf0408f53151dcc31eb43c3bd10a9fb0d7ba32b95b...,4250334444,Guraj93,24,guraj_hots,Harbinger,1,0,0,1,1
18,5,False,False,Ryphe,100,Occultist,b6387a6be18939980798db2ec8e8a2924027fa803e063a...,4250334444,Ryphe,40,rypherious,Harbinger,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59757,14996,False,True,iTrofiiforTi,94,Ascendant,575bcf4246a6287f2cd19f3749bb9e803cd59b1f378cc4...,2720826701,troffi,40,NaN,Harbinger,1,1,1,1,3
59761,14997,False,False,Semirf,94,Chieftain,88f31e056e4ea4c56c1fb5a4a6b5e02ba1ec093ffa028c...,2720815490,priima,26,semikreisi,Harbinger,1,1,0,0,1
59766,14998,False,False,Vicoachh,94,Raider,8f55b48a26bfaf96bb0f5cebf6af76b27ad2eedd6dbdd6...,2720813347,ViCoach,28,NaN,Harbinger,1,0,0,1,1
59770,14999,False,False,Imastunboy,94,Berserker,5f6c5f759dc46edfcfd6292c792a8dcd33bada2bf070b4...,2720777728,Gangbash,22,NaN,Harbinger,1,1,0,0,1


In [42]:
df_Har['rank'].nunique()

14918

rank 는 15000 까지 있지만 row는 12918 로 일부 유실되어있다. 이는 df_Har 뿐만이 아닐 것이다.

어떠한 이유에서든 없는 것은 지금 다시 가져올 수 없으므로, 빠지는 값이 없이 해주는 것이 더 정확한 계산을 가져올 수 있을 것이라 생각한다.

### Insert missing values in 'rank'

구분해주기 위해 기존의 rank를 rank를 덮는다.

In [43]:
# 기존 rank에 rank를 다시 매긴다. ascending = True : 작은 순대로 오름차순 1부터 순위 매김.

df_Har['rank'] = df_Har['rank'].rank(ascending = True)
df_Har

/var/folders/qw/j72d9nwn5dq0vxtqtw2bbtnh0000gp/T/ipykernel_1047/257618127.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Har['rank'] = df_Har['rank'].rank(ascending = True)


,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
0,1.0,False,False,Tzn_NecroIsFineNow,100,Necromancer,3dcddd59f5088893f734f39686350990dae168cc4f4b00...,4250334444,TheTzn,40,thetzn,Harbinger,1,0,1,0,1
4,2.0,False,False,Cool_NecroIsFineNow,100,Deadeye,24ae924ceed7989ef3d3d6772612832bb467a6094352c7...,4250334444,cooltail,22,NaN,Harbinger,1,0,0,1,1
8,3.0,False,False,Rami_The_Negromaniac,100,Necromancer,00c0030fc548cc4c0d804564f8fb356610044b7e6d484f...,4250334444,rami1337,40,rami1337,Harbinger,1,0,1,0,1
13,4.0,False,False,Guraj_NecroIsFineNow,100,Raider,34bedf0408f53151dcc31eb43c3bd10a9fb0d7ba32b95b...,4250334444,Guraj93,24,guraj_hots,Harbinger,1,0,0,1,1
18,5.0,False,False,Ryphe,100,Occultist,b6387a6be18939980798db2ec8e8a2924027fa803e063a...,4250334444,Ryphe,40,rypherious,Harbinger,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59757,14914.0,False,True,iTrofiiforTi,94,Ascendant,575bcf4246a6287f2cd19f3749bb9e803cd59b1f378cc4...,2720826701,troffi,40,NaN,Harbinger,1,1,1,1,3
59761,14915.0,False,False,Semirf,94,Chieftain,88f31e056e4ea4c56c1fb5a4a6b5e02ba1ec093ffa028c...,2720815490,priima,26,semikreisi,Harbinger,1,1,0,0,1
59766,14916.0,False,False,Vicoachh,94,Raider,8f55b48a26bfaf96bb0f5cebf6af76b27ad2eedd6dbdd6...,2720813347,ViCoach,28,NaN,Harbinger,1,0,0,1,1
59770,14917.0,False,False,Imastunboy,94,Berserker,5f6c5f759dc46edfcfd6292c792a8dcd33bada2bf070b4...,2720777728,Gangbash,22,NaN,Harbinger,1,1,0,0,1


In [52]:
# 정수 타입으로 바꾸어준다.

df_Har['rank'] = df_Har['rank'].astype('int64', copy = True)

/var/folders/qw/j72d9nwn5dq0vxtqtw2bbtnh0000gp/T/ipykernel_1047/766561525.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Har['rank'] = df_Har['rank'].astype('int64', copy = True)


In [53]:
# checking

df_Har

,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
0,1,False,False,Tzn_NecroIsFineNow,100,Necromancer,3dcddd59f5088893f734f39686350990dae168cc4f4b00...,4250334444,TheTzn,40,thetzn,Harbinger,1,0,1,0,1
4,2,False,False,Cool_NecroIsFineNow,100,Deadeye,24ae924ceed7989ef3d3d6772612832bb467a6094352c7...,4250334444,cooltail,22,NaN,Harbinger,1,0,0,1,1
8,3,False,False,Rami_The_Negromaniac,100,Necromancer,00c0030fc548cc4c0d804564f8fb356610044b7e6d484f...,4250334444,rami1337,40,rami1337,Harbinger,1,0,1,0,1
13,4,False,False,Guraj_NecroIsFineNow,100,Raider,34bedf0408f53151dcc31eb43c3bd10a9fb0d7ba32b95b...,4250334444,Guraj93,24,guraj_hots,Harbinger,1,0,0,1,1
18,5,False,False,Ryphe,100,Occultist,b6387a6be18939980798db2ec8e8a2924027fa803e063a...,4250334444,Ryphe,40,rypherious,Harbinger,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59757,14914,False,True,iTrofiiforTi,94,Ascendant,575bcf4246a6287f2cd19f3749bb9e803cd59b1f378cc4...,2720826701,troffi,40,NaN,Harbinger,1,1,1,1,3
59761,14915,False,False,Semirf,94,Chieftain,88f31e056e4ea4c56c1fb5a4a6b5e02ba1ec093ffa028c...,2720815490,priima,26,semikreisi,Harbinger,1,1,0,0,1
59766,14916,False,False,Vicoachh,94,Raider,8f55b48a26bfaf96bb0f5cebf6af76b27ad2eedd6dbdd6...,2720813347,ViCoach,28,NaN,Harbinger,1,0,0,1,1
59770,14917,False,False,Imastunboy,94,Berserker,5f6c5f759dc46edfcfd6292c792a8dcd33bada2bf070b4...,2720777728,Gangbash,22,NaN,Harbinger,1,1,0,0,1


### also, remained 'ladders'

In [43]:
# 기존 rank에 rank를 다시 매긴다. ascending = True : 작은 순대로 오름차순 1부터 순위 매김.

df_Har['rank'] = df_Har['rank'].rank(ascending = True)
df_Har

/var/folders/qw/j72d9nwn5dq0vxtqtw2bbtnh0000gp/T/ipykernel_1047/257618127.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Har['rank'] = df_Har['rank'].rank(ascending = True)


,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
0,1.0,False,False,Tzn_NecroIsFineNow,100,Necromancer,3dcddd59f5088893f734f39686350990dae168cc4f4b00...,4250334444,TheTzn,40,thetzn,Harbinger,1,0,1,0,1
4,2.0,False,False,Cool_NecroIsFineNow,100,Deadeye,24ae924ceed7989ef3d3d6772612832bb467a6094352c7...,4250334444,cooltail,22,NaN,Harbinger,1,0,0,1,1
8,3.0,False,False,Rami_The_Negromaniac,100,Necromancer,00c0030fc548cc4c0d804564f8fb356610044b7e6d484f...,4250334444,rami1337,40,rami1337,Harbinger,1,0,1,0,1
13,4.0,False,False,Guraj_NecroIsFineNow,100,Raider,34bedf0408f53151dcc31eb43c3bd10a9fb0d7ba32b95b...,4250334444,Guraj93,24,guraj_hots,Harbinger,1,0,0,1,1
18,5.0,False,False,Ryphe,100,Occultist,b6387a6be18939980798db2ec8e8a2924027fa803e063a...,4250334444,Ryphe,40,rypherious,Harbinger,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59757,14914.0,False,True,iTrofiiforTi,94,Ascendant,575bcf4246a6287f2cd19f3749bb9e803cd59b1f378cc4...,2720826701,troffi,40,NaN,Harbinger,1,1,1,1,3
59761,14915.0,False,False,Semirf,94,Chieftain,88f31e056e4ea4c56c1fb5a4a6b5e02ba1ec093ffa028c...,2720815490,priima,26,semikreisi,Harbinger,1,1,0,0,1
59766,14916.0,False,False,Vicoachh,94,Raider,8f55b48a26bfaf96bb0f5cebf6af76b27ad2eedd6dbdd6...,2720813347,ViCoach,28,NaN,Harbinger,1,0,0,1,1
59770,14917.0,False,False,Imastunboy,94,Berserker,5f6c5f759dc46edfcfd6292c792a8dcd33bada2bf070b4...,2720777728,Gangbash,22,NaN,Harbinger,1,1,0,0,1


In [52]:
# 정수 타입으로 바꾸어준다.

df_Har['rank'] = df_Har['rank'].astype('int64', copy = True)

/var/folders/qw/j72d9nwn5dq0vxtqtw2bbtnh0000gp/T/ipykernel_1047/766561525.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Har['rank'] = df_Har['rank'].astype('int64', copy = True)


In [53]:
# checking

df_Har

,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
0,1,False,False,Tzn_NecroIsFineNow,100,Necromancer,3dcddd59f5088893f734f39686350990dae168cc4f4b00...,4250334444,TheTzn,40,thetzn,Harbinger,1,0,1,0,1
4,2,False,False,Cool_NecroIsFineNow,100,Deadeye,24ae924ceed7989ef3d3d6772612832bb467a6094352c7...,4250334444,cooltail,22,NaN,Harbinger,1,0,0,1,1
8,3,False,False,Rami_The_Negromaniac,100,Necromancer,00c0030fc548cc4c0d804564f8fb356610044b7e6d484f...,4250334444,rami1337,40,rami1337,Harbinger,1,0,1,0,1
13,4,False,False,Guraj_NecroIsFineNow,100,Raider,34bedf0408f53151dcc31eb43c3bd10a9fb0d7ba32b95b...,4250334444,Guraj93,24,guraj_hots,Harbinger,1,0,0,1,1
18,5,False,False,Ryphe,100,Occultist,b6387a6be18939980798db2ec8e8a2924027fa803e063a...,4250334444,Ryphe,40,rypherious,Harbinger,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59757,14914,False,True,iTrofiiforTi,94,Ascendant,575bcf4246a6287f2cd19f3749bb9e803cd59b1f378cc4...,2720826701,troffi,40,NaN,Harbinger,1,1,1,1,3
59761,14915,False,False,Semirf,94,Chieftain,88f31e056e4ea4c56c1fb5a4a6b5e02ba1ec093ffa028c...,2720815490,priima,26,semikreisi,Harbinger,1,1,0,0,1
59766,14916,False,False,Vicoachh,94,Raider,8f55b48a26bfaf96bb0f5cebf6af76b27ad2eedd6dbdd6...,2720813347,ViCoach,28,NaN,Harbinger,1,0,0,1,1
59770,14917,False,False,Imastunboy,94,Berserker,5f6c5f759dc46edfcfd6292c792a8dcd33bada2bf070b4...,2720777728,Gangbash,22,NaN,Harbinger,1,1,0,0,1


In [ ]:
df_byladder['class'].unique()

In [ ]:
df_byladder.groupby(by = ['class']).mean()

## Kinds of Path of Exile classes

해당 데이터셋에서, poe에는 ['Necromancer', 'Deadeye', 'Raider', 'Occultist', 'Pathfinder',
       'Guardian', 'Champion', 'Gladiator', 'Inquisitor', 'Elementalist',
       'Slayer', 'Berserker', 'Juggernaut', 'Ascendant', 'Trickster',
       'Hierophant', 'Chieftain', 'Saboteur', 'Assassin', 'Ranger',
       'Marauder', 'Scion', 'Witch', 'Duelist', 'Shadow', 'Templar']  총 24개의 unique 클래스가 존재하며,

POE 공식 사이트를 참고한 결과 각 직업별 클래스는 이렇게 나뉘고 있다.

    Duelist - 'Slayer', 'Gladiator', 'Champion'
    Shadow - 'Assassin', 'Saboteur', 'Trickster'
    Marauder(약탈자 라고 한다) - 'Juggernaut', 'Berserker', 'Chieftain',
    Witch - 'Necromancer', 'Elementalist', 'Occultist'
    Ranger - 'Raider', 'Deadeye', 'Pathfinder'
    Templar - 'Inquisitor', 'Hierophant', 'Guardian'
    Scion - 'Ascendant'
    
일반적으로 레인저는 원딜러, 듀얼리스트는 전사로 보이며, 위치는 마딜러, 템플러는 수도승 같은 등등 다른 게임에서 보이는 직업과 비슷한 부분이 있지만,

각 클래스가 가진 특성들도 다르기 때문에 poe에서 제공하고있는 클래스 별로도 나눌 수 있을 것이고, 일반적인 rpg 특성에 따라 이를 나누는 방법은 이런 방식들이 될 것이다.

    1. 탱커, 딜러, 힐러(보조직업)
    2. 탱커, 일반딜러, 암살딜러, 힐러(보조직업)
    2. 탱커, 근접딜러, 원거리딜러, 힐러(보조직업)
    3. 탱커, 근접딜러, 원거리물리딜러, 원거리마법딜러, 힐러(보조직업)
    4. 근거리, 원거리, 마법, 힐러(보조직업)
    5. 점사딜스킬위주, 광역딜스킬위주, 힐러(보조직업) ... 등등

이에 따라 각 클래스를 읽어보고 데이터 labeling을 진행하도록 한다.

스킬 범위나 세부 효과에 관련한 부분은 실제로 poe를 해본 사람이 분석하는 것이 더 바를 것 같다는 판단이 든다.

클래스의 호불호 혹은 성능을 떠나서, 세부적인 수치와 관련해서는 플레이 해보지 않는 이상 체감을 해볼 수 없기 때문이다.

예를 들어 템플러의 가디언은 방어 버프 혹은 적의 버프를 방해하는 스킬들이 있는데 상세히 알기는 어렵다.

---

https://hong-sam.tistory.com/27 글을 참조하여 일반적인 직업으로 나누는 것을 먼저 하기로 결정.

글에 따르면 
```
    Duelist : 빠른 근접 전투(힘, 민첩)
    Shadow : 민첩하고 지능적인 전투(민첩, 지능)
    Marauder : 전사(힘)
    Witch : 마법사(지능)
    Ranger : 궁수(민첩)
    Templar : 성기사(힘, 지능)
    Scion : 모든 스탯이 중간(힘, 민첩, 지능)
```

으로 나누어져있다. 사이온 같은 경우 성장 중 사이온을 구하는 퀘스트를 깨야 직업을 할 수 있다고 한다.

스킬을 찍는 데에 자유로운 것이 특징이나 자신의 원래 직업과 스탯에 따라 가는 것이 더 좋은 효율을 낼 수 있다는 점에서 다른 게임과 크게 다르지는 않다.

살펴본 결과 패시브 혹은 버프(디버프)스킬들도 많이 포함되어 있었다. 

자유도가 높지만 높아도 그 안에서 더 좋은 효율을 내는 일부 직업이나 스킬트리가 유행하여 해당 직업에는 한 클래스가 주를 이루기 마련인데, 그런 경향도 있는지 살펴보아야 겠다.

문제는 사이온인데, 사람마다 찍는 유형에 따라 근접일 수도 원거리일 수도, 완전히 보조 일수도, 그 중간 어딘가일 수도 있다.

여러 능력의 조합을 사용하는 것의 이점과 단점은 극명하기 때문에 숙련된 사람이 아니면 사이온을 달고 랭킹에 진입하기는 쉽지 않았을 것이라는 생각도 든다.


방식 1: 세부 클래스 전 직업(전직 전)으로 묶은 다음 그에 따라 labeling, 사이온은 사이온으로.
```
    근 - Duelist - 'Slayer', 'Gladiator', 'Champion'
    근 - Shadow - 'Assassin', 'Saboteur', 'Trickster'
    근 - Marauder - 'Juggernaut', 'Berserker', 'Chieftain',
    원 - Witch - 'Necromancer', 'Elementalist', 'Occultist'
    원 - Ranger - 'Raider', 'Deadeye', 'Pathfinder'
    원 - Templar - 'Inquisitor', 'Hierophant', 'Guardian'
    사이온 - Scion - 'Ascendant'
```
방식 2: 세부 클래스(전직 후)에 따라 labeling (딜 위주, 탱킹 위주, 보조 위주 - 게임 공부를 더 하고 나누는 걸로. EDA 하다보면 아이디어를 얻을수도 있고,)

    'Slayer' : 양손 무기, 생존력 특화 Duelist
    'Gladiator' : 쌍수 혹은 한손-방패 Duelist
    'Champion' : 적 디버프, 자신 버프 Duelist
    'Assassin' : 출혈, 독 등 이용한 치명 딜 위주 Shadow
    'Saboteur' : 트랩, 마인 활용 Shadow
    'Trickster' : 기동성, 회피, 공격속도 위주 Shadow
    'Juggernaut' : 탱킹 중심 Marauder
    'Berserker' : 흡혈이나 피해 비례 데미지 등 광전사 Marauder
    'Chieftain' : 화염, 적 디버프 이용 Marauder
    'Necromancer' : 소환술 특화 Witch
    'Elementalist' : 원소 피해 특화 Witch
    'Occultist' : 카오스 피해, 저주 특화 Witch
    'Raider' : 회피, 처치 후 얻는 버프를 이용하는 Ranger
    'Deadeye' : 치명타, 딜 중심 Ranger
    'Pathfinder' : 플라스크(물약), 치유효과 강화, 디버프 제거, 원소, 카오스 이용 Ranger
    'Inquisitor' : 원소 치명타, 마나 재생, 자신 버프 Templer
    'Hierophant' : 토템 및 마나 변환, 받은 피해 마나로 감소 등의 Templer
    'Guardian' : 서포트 및 보호(방어버프, 디버프제거, 치유 등) 특화 Templer
    'Ascendant' : 타 클래스들의 조합하는 Scion. 성능은 다소 떨어질 수 있음.

방식 3: 특정 스탯 한 가지 / 혹은 두 가지 / 세 가지

    'Marauder', 'Witch', 'Ranger'
    'Duelist', 'Shadow', 'Templar'
    'Scion'

In [ ]:
df_byladder['class'].unique()

전직 전과 전직 후 *모두 섞여있는 것* 또한 볼 수 있다. 이에 따라 이를 기준으로 나눌 수도 있다. 
전직 전과 후로 찍을 수 있는 스킬이 다를 것이기 때문이다. (물론 젬이 필요하다고 했기 때문에 그것이 없어서 그 직업을 운용중일 지도 모른다)

# class labeling

## ['special_class']: before and after to select special classes

In [ ]:
df

In [ ]:
df['special_class'] = 0

In [ ]:
pre_classes = ['Duelist', 'Shadow', 'Marauder', 'Witch', 'Ranger', 'Templar', 'Scion']

In [ ]:
df['special_class'] = np.where(df['class'].isin(pre_classes), 0, 1)

In [ ]:
df.head(10)

### How many ranked users in there who didn't select the special class?

In [ ]:
df['special_class'].value_counts()

In [ ]:
# 위와 같은 결과를 그룹바이로 낼 수 있다.

freq = df.groupby(['special_class']).size() 
print(freq)

### Then, what pre-classes of them are there?

In [ ]:
df[df['special_class'] == 0]['class'].value_counts()

위 결과를 보니 더더욱! 라벨링을 하는게 좋겠다는 생각이 드는데, 게임을 어떤 관점에서 볼지에 따라 나누는 방식을 달리할 수 있을 것이다.

포지션 자체인지 아니면 전체적인 스킬의 유형(사거리 등)에 관한 것인지 사람마다 중요시 하는 부분이 다르기 때문에... 

살펴보면 살펴볼 수록 이 게임은 사거리 자체보다는 포지션이나, 솔플 난이도 여부에 따라 갈리는게 나을 것 같다는 생각이 든다. 그래서 SSF 모드가 따로 있는 것이기도 한 이유일 것이다.

### What play modes of them are there?

In [ ]:
df[df['special_class'] == 0]['ladder'].value_counts()

독특하게도, 전직하지 않은 사람들은 일반 Harbinger 에는 없었고, 대부분 솔로 플레이모드에 많았다.

아무래도 전직이라는 건 세부 직업을 고르는 것, 즉 특화를 하는 것이기 때문에 특화되지 않은 다른 면에서는 부족해질 수 있다고 생각해보면, 

솔로플레이에서는 전직하지 않은 직업으로 플레이 하는게 더 나을 수도 있다는 생각이다. 물론 스킬을 찍는 포인트가 한정되어있다는 사실 안에 말이다.



###  df_pre_cl : stats description by whether pre or post special classes

전직을 하지 않은 사람들과 한 사람들의 통계적 요약을 보고자 한다.

In [ ]:
df_pre_cl = df[df['special_class'] == 0]
df_pre_cl

In [ ]:
df_pre_cl.describe()

평균 랭크 자체는 높은 편이아니고, 레벨의 최대 값은 94였다. 최대 레벨이 100인 점을 감안할 때 물음표가 생길 수 밖에 없는 부분이다. 게임을 정확히 이해하지 않고서는 풀기 어려운 의문인 것 같다.

반대로 전직을 한 사람들을 보자면, 

In [ ]:
df[df['special_class'] == 1].describe()

세부 클래스로 전직한 평균 랭크가 약 3천 이상 더 높고, 레벨도 평균적으로 더 높다. 

하지만 경험치는 그들보다 평균적으로 더 적었고, 시도도 더 많이 한 것으로 나타났다.

이를 토대로 합리적의심을 해보자면, 숙련도나 게임 경력에 따라서 전직 여부를 결정하지는 않았을 것이라고 추측해 볼 수 있다.

## ['stat_Str', 'stat_Int', 'stat_Dex']: columns by kind of stats and stats numbers

먼저, 특정 스탯 한 가지 / 혹은 두 가지 / 세 가지

    'Marauder', 'Witch', 'Ranger'
    'Duelist', 'Shadow', 'Templar' 
    'Scion'

에 따라 stat_num(개수) 을 1, 2, 3으로 나누게 될 것이다.

각 클래스에 따라 어떤 특성을 찍었는지 특성(힘, 민첩, 지능) 을 나누어서 여부를 라벨링 한 다음 sum 으로 낼 계획


In [ ]:
# 각 클래스 나누기 (전직하지 않은 사람들도 있기 때문에 각 전직 이전 직업도 넣어준다.)

# 한 개인 직업
Marauder = ['Marauder', 'Juggernaut', 'Berserker', 'Chieftain'] # Strength
Witch = ['Witch', 'Necromancer', 'Elementalist', 'Occultist'] # Intelligence
Ranger = ['Ranger', 'Raider', 'Deadeye', 'Pathfinder'] # Dexterity

# 두 개인 직업
Duelist = ['Duelist', 'Slayer', 'Gladiator', 'Champion'] # Strength, Dexterity
Shadow = ['Shadow', 'Assassin', 'Saboteur', 'Trickster'] # Dexterity, Intelligence
Templar = ['Templar', 'Inquisitor', 'Hierophant', 'Guardian'] # Intelligence, Strength

# 세 개인 직업
Scion = ['Scion', 'Ascendant'] # Strength, Dexterity, Intelligence

In [ ]:
# 앞에 stat을 굳이 붙인 이유는 다 쓰긴 길고 자르자니 str, int 이런 문자여서 보기가 애매하여 붙였다.

df['stat_Str'] = 0
df['stat_Int'] = 0
df['stat_Dex'] = 0

In [ ]:
# Strength
df['stat_Str'] = np.where(df['class'].isin(['Marauder', 'Juggernaut', 'Berserker', 'Chieftain', 
                                            'Duelist', 'Slayer', 'Gladiator', 'Champion', 
                                            'Templar', 'Inquisitor', 'Hierophant', 'Guardian',
                                            'Scion', 'Ascendant']), 1, 0)

# Intelligence
df['stat_Int'] = np.where(df['class'].isin(['Witch', 'Necromancer', 'Elementalist', 'Occultist', 
                                            'Shadow', 'Assassin', 'Saboteur', 'Trickster',
                                            'Templar', 'Inquisitor', 'Hierophant', 'Guardian',
                                            'Scion', 'Ascendant']), 1, 0)

# Dexterity
df['stat_Dex'] = np.where(df['class'].isin(['Ranger', 'Raider', 'Deadeye', 'Pathfinder',
                                            'Duelist', 'Slayer', 'Gladiator', 'Champion',
                                            'Shadow', 'Assassin', 'Saboteur', 'Trickster',
                                            'Scion', 'Ascendant']), 1, 0)

In [ ]:
df.sample(20)

#### checking

In [ ]:
# check
# 위에서 만들었던 리스트를 리스트 안에 넣는다.
pre_classes_list = [Marauder, Witch, Ranger, Duelist, Shadow, Templar, Scion]


for i in pre_classes_list:
    print("Class names: ", i)
    print("Class nums: ", df['class'].isin(i).sum())
    print(df[df['class'].isin(i)][['stat_Str','stat_Int', 'stat_Dex']].sum())
    print("================================================================")

제대로 들어간 것을 확인했다. 

## ['stats_num']: How many stats they set on main?

다음은 주스탯 컬럼의 여부에 따라서 능력치를 몇 가지를 사용하고 있는지를 따져볼 차례이다.

In [ ]:
df.sample(5)

In [ ]:
# stats_num is the number of main stats for the class

df['stats_num'] = df['stat_Str'] + df['stat_Int'] + df['stat_Dex']

# checking

df.sample(5)

#### plus...

클래스들의 특성에 따라 광딜, 도트딜, 강화약화주력 이런식으로 나눠볼까 했는데 역시 게임을 직접해보지 않고서야 아무리 tier들을 읽어보아도(https://gamerant.com/best-path-of-exile-ascendancy-classes-ranked/) 이해할 수 없으므로 이렇게만 나누고 분석을 시작해보고자 한다.

이후 알게되면서 필요하다면 추가 라벨링을 통해 분석을 할 예정이다.

# data export

다른 파일에서 이어서 쓰기 위해서 내보내려고 한다. 

In [ ]:
df.to_csv("poe_stats_1preprocessing.csv", index = False)

In [ ]:
# load to check 
df_1 = pd.read_csv("poe_stats_1preprocessing.csv")
df_1